In [ ]:
!pip install yfinance

In [ ]:
!pip install pandas_ta

In [33]:
import numpy as np
import pandas as pd
import yfinance as yf
import pandas_datareader.data as web
import pandas_ta as ta
import matplotlib.pyplot as plt
from datetime import date
plt.style.use('fivethirtyeight')
yf.pdr_override()

In [ ]:
stocksymbols = ['HDFCBANK.NS']
startdate = date(2018,4,1)
end_date = date(2019,3,31)
print(end_date)
def getMyPortfolio(stocks = stocksymbols ,start = startdate , end = end_date):
    data = web.get_data_yahoo(stocks , data_source='yahoo' , start = start ,end= end )
    return data

In [ ]:
df = getMyPortfolio(stocksymbols)


In [36]:
CustomStrategy = ta.Strategy(
    name="Defining Various TI to be used",
    description="SMA 30,70, BBANDS, RSI, MACD, STOCH, EMA 30,50,100",
    ta=[
        {"kind": "sma", "length": 30},
        {"kind": "sma", "length": 70},
        {"kind": "bbands", "length": 20},
        {"kind": "rsi"},
        {"kind": "macd", "fast": 8, "slow": 21},
        {"kind": "stoch"},
        {"kind": "ema", "length": 30},
        {"kind": "ema", "length": 50},
        {"kind": "ema", "length": 100},
    ]
)
df.ta.strategy(CustomStrategy)

In [37]:
df.rename(columns = {'EMA_30':'Short', 'EMA_50':'Middle', 'EMA_100':'Long'}, inplace = True)

In [ ]:
df.head(200)

In [ ]:
plt.plot(df['SMA_30'], label='SMA', color='green')
plt.plot(df['Open'], label='SHARE PRICE', color='black')

#add legend
plt.legend(title='Group')

#add axes labels and a title
plt.ylabel('SHARE PRICE', fontsize=14)
plt.xlabel('Time', fontsize=14)
plt.title('TA', fontsize=16)

#display plot
plt.show()

In [40]:
def buy_sell_function(data):
  buy_list = []
  sell_list = []
  flag_long = False
  flag_short = False
  for i in range(0,len(data)):
    if data['Middle'][i] < data['Long'][i] and data['Middle'][i] > data['Short'][i] and flag_long == False and flag_short == False:
        buy_list.append(data['Close'][i])
        sell_list.append(np.nan)
        flag_short = True
    elif data['Middle'][i] > data['Long'][i] and data['Middle'][i] < data['Short'][i] and flag_short == False and flag_long == False:
        buy_list.append(data['Close'][i])
        sell_list.append(np.nan)
        flag_long = True
    elif flag_short == True and data['Short'][i] > data['Middle'][i]:
        sell_list.append(data['Close'][i])
        buy_list.append(np.nan)
        flag_short = False
    elif flag_long == True and data['Short'][i] < data['Middle'][i]:
        sell_list.append(data['Close'][i])
        buy_list.append(np.nan)
        flag_long = False
    else:
        buy_list.append(np.nan)
        sell_list.append(np.nan)
  return (buy_list, sell_list)

df['Buy'] = buy_sell_function(df)[0]
df['Sell'] =buy_sell_function(df)[1]

In [ ]:
df.head(200)

In [41]:
bs=df['Buy']
ss=df['Sell']
b = bs[np.logical_not(np.isnan(bs))]
s = ss[np.logical_not(np.isnan(ss))]
print(b)
print(s)

Date
2018-08-21    1048.025024
2018-10-10     983.575012
2018-12-05    1049.599976
Name: Buy, dtype: float64
Date
2018-09-07    1028.474976
2018-11-29    1065.525024
Name: Sell, dtype: float64


In [42]:
tp=0
fl=0
for i in range(len(s)):
  tp = tp + s[i]
  tp = tp - b[i]
print(tp)

62.39996337890625
